# Advection Diffusion Solver 


## Problem statement

We are investigating the Advection Diffusion Equation
$$
  \frac{\partial u^{\mathrm{MS}}}{\partial t} +\vec{u} \cdot \nabla u^{\mathrm{MS}} - \mu \nabla^2 u^{\mathrm{MS}} + S(u)=0 \in (0,2)^2
$$
with the initial value
$$
    u_0(x,y) = \sin(a x) \sin(b y),
$$
and the manufactured solution
$$
u^{\mathrm{MS}} = \sin(a x) \sin(b y) \cos(c t)
$$

#### Calculation of the Source Term
calcluate all parts
$$ 
\frac{\partial u^{\mathrm{MS}}}{\partial t} = -c \sin(a x) \sin(b y) \sin(c t)= -c\frac{\sin(c t)}{\cos(c t)} u^{\mathrm{MS}}
$$
$$
\vec{u} \cdot \nabla u^{\mathrm{MS}} = \cos(ct) [u_1 a \cos(ax) \sin(by)+ u_2 b \sin(ax) \cos(by)] =  u_1 a  \frac{\cos(ax)}{\sin(a x)}  u^{\mathrm{MS}}+ u_2 b \frac{\cos(by)}{\sin(by)}  u^{\mathrm{MS}}
$$
$$
\mu \nabla^2 u^{\mathrm{MS}} = \mu \cos(ct)[-a^2 \sin(ax) \sin(by)  - b^2 \sin(ax) \sin(by)] =  \mu (-a^2 - b^2)u^{\mathrm{MS}}
$$

Putting that together we obtain the source term as
$$
S(u) = -(\frac{\partial u^{\mathrm{MS}}}{\partial t} +\vec{u} \cdot \nabla u^{\mathrm{MS}} - \mu \nabla^2 u^{\mathrm{MS}}) 
$$


In [162]:
#r ".\binaries\\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 204
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#163.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [163]:
using BoSSS.Solution.XdgTimestepping;

## Creation of Differential Operator


Velocity field and Exact solution for Dirichlet Boundary

#### Calculation of the Source Term
calcluate all parts
$$ 
\frac{\partial u^{\mathrm{MS}}}{\partial t} = -c \sin(a x) \sin(b y) \sin(c t)= -c\frac{\sin(c t)}{\cos(c t)} u^{\mathrm{MS}}
$$
$$
\vec{u} \cdot \nabla u^{\mathrm{MS}} = \cos(ct) [u_1 a \cos(ax) \sin(by)+ u_2 b \sin(ax) \cos(by)] =  u_1 a  \frac{\cos(ax)}{\sin(a x)}  u^{\mathrm{MS}}+ u_2 b \frac{\cos(by)}{\sin(by)}  u^{\mathrm{MS}}
$$
$$
\mu \nabla^2 u^{\mathrm{MS}} = \mu \cos(ct)[-a^2 \sin(ax) \sin(by)  - b^2 \sin(ax) \sin(by)] =  \mu (-a^2 - b^2)u^{\mathrm{MS}}
$$

Putting that together we obtain the source term as
$$
S(u) = -(\frac{\partial u^{\mathrm{MS}}}{\partial t} +\vec{u} \cdot \nabla u^{\mathrm{MS}} - \mu \nabla^2 u^{\mathrm{MS}}) 
$$


In [164]:
using static System.Math;
public static class MyGlobals {
    public static double u(double[] X) => 0.8;
    public static double v(double[] X) => 0.6;
    public static double a = 2.3;
    public static double b = 2.9;
    public static double c = 4.2;
    public static double nu=0.5;

    public static double uExact(double[] X, double t) => Sin(a * X[0]) * Sin(b * X[1]) * Cos(c * t);

    public static double TimeDerivative(double[] X, double t) => -c * Sin(a * X[0]) * Sin(b * X[1]) * Sin(c * t);

    public static double ConvectiveTerm(double[] X, double t) => 
    Cos(c * t) * (u(X) * a * Cos(a * X[0]) * Sin(b * X[1]) + v(X) * b * Sin(a * X[0]) * Cos(b * X[1]));

    public static double DiffusionTerm(double[] X, double t) => nu * Cos(c * t) * (-a*a * Sin(a * X[0]) * Sin(b * X[1]) - b*b * Sin(a * X[0]) * Sin(b * X[1]));

     public static double SourceTerm (double[] X, double t) =>  -(TimeDerivative(X,t)+ConvectiveTerm(X,t)-DiffusionTerm(X,t));
  
//     public static double TimeDerivative_u(double U,double[] X, double t) => -c *  Sin(c * t)*U/Cos(c*t);

//     public static double ConvectiveTerm_u(double U,double[] X, double t) => 
//    (u(X) * a * Cos(a * X[0]) /Sin(a*X[0])+ v(X) * b * Cos(b * X[1])/Sin(b*X[1]) )*U;

//     public static double DiffusionTerm_u(double U,double[] X, double t) => nu *(-a*a-b*b)*U;
//     // Calculate the corresponding source term S(u)
//     public static double S_u (double U,double[] X, double t) =>  -(TimeDerivative_u(U,X,t)+ConvectiveTerm_u(U,X,t)-DiffusionTerm_u(U,X,t));
   
}

Regarding the spatial part of the operator, we re-use the SIP-implementation 
which is already available in the `BoSSS.Solution.NSECommon` library.
For this implementation, whe have to specify the diffusion coefficient (here to -0.5)
as well as the Location of the Dirichlet boundary.

In [165]:
class Laplace : BoSSS.Solution.NSECommon.SIPLaplace {

    public Laplace() : base(1.4, "u") { } // override the constuctor; 
    //                                       The factor 1.4 is a constant "safty factor" multiper for the penalty parameter.
    //                                       The base implementation takes care about all other penalty factor dependencies, 
    //                                       i.e. dependence of the penatly factor on local cell size, DG polynomial degree and cell shape.
    //                                       The second argument, "u", is the name of the variable for the `ArgumentOrdering`.


    // Specifies whetherss a specific point (`inp.X`) is either a Dirichlet or a Neumann boundary;
    // Since in our case ther, the entire boundary should be Diriclet, we always return true.
    protected override bool IsDirichlet(ref CommonParamsBnd inp) {
        return true;
    }
    
    // diffusion coefficient
    override public double Nu(double[] x, double[] p, int jCell) {
        return -0.5;
    }
    override protected double g_Diri(ref CommonParamsBnd inp) { 
        //Console.WriteLine("Evaluated at x=" + inp.X);
        return MyGlobals.uExact(inp.X,inp.time); }


}

#### Advection Operator

The flux we are considering is the local Lax-Friedrichs flux
$$
\hat{f}(c_h^-, c_h^+, \vec{u} \cdot \vec{n}) = \overline{\vec{u} \, c_h} \cdot \vec{n} + \frac{C}{2} \lbrack \lbrack {c_h} \rbrack \rbrack.
$$
The constant $C \in \mathbb{R}^+$ has to be sufficiently large in order to guarantee the stability of the 
numerical scheme. We choose 
$$
C = \max \vert{\vec{u} \cdot \vec{n}}\vert 
$$


In [166]:
//double C = 0.8;     
 
Func<double, double, Vector, Vector, double> laxFriedrichsFlux =     
    delegate(double Uin, double Uout, Vector n, Vector velocityVector) {     
        var C=Math.Abs(velocityVector*n);
        return 0.5 * (Uin + Uout) * velocityVector * n - C * (Uout - Uin);     
    };

In [167]:

class ScalarTransportFlux : NonlinearFlux {     
 
    private Func<double, double, Vector, Vector, double> numericalFlux;     
 
    // Provides instances of this class with a specific flux implementation     
    public ScalarTransportFlux(     
        Func<double, double, Vector, Vector, double> numericalFlux) {     
 
        this.numericalFlux = numericalFlux;     
    }     
 
    public override IList<string> ArgumentOrdering {     
        get { return new string[] { "u" }; }     
    }     
 
    protected override void Flux(     
        double time, double[] x, double[] U, double[] output) {     
 
        output[0] = MyGlobals.u(x) * U[0];     
        output[1] = MyGlobals.v(x) * U[0];     
    }     
 
    // Makes use of the flux implementation supplied in the constructor     
    protected override double InnerEdgeFlux(     
        double time, double[] x, double[] normal,      
        double[] Uin, double[] Uout, int jEdge) {     
 
        Vector n              = new Vector(normal);     
        Vector velocityVector = new Vector(MyGlobals.u(x), MyGlobals.v(x));     
 
        return numericalFlux(Uin[0], Uout[0], n, velocityVector);     
    }     
 
    protected override double BorderEdgeFlux(     
        double time, double[] x, double[] normal, byte EdgeTag,     
        double[] Uin, int jEdge) {     
 
        double[] Uout = new double[] { MyGlobals.uExact(x, time) };     
 
        return InnerEdgeFlux(time, x, normal, Uin, Uout, jEdge);     
    }     
}

In [168]:
class SourceTerm : IVolumeForm
{

    public TermActivationFlags VolTerms => TermActivationFlags.UxV;

    public IList<string> ArgumentOrdering {     
        get { return new string[] { "u" }; }     
    } 

    public IList<string> ParameterOrdering => null;

    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV)
    {
        //return U[0]*V;
        //return MyGlobals.S_u(U[0],cpv.Xglobal,cpv.time)*V;
        return MyGlobals.SourceTerm(cpv.Xglobal,cpv.time)*V;
    }
}

#### Operator assembly
We compose the differential operator from the previously created 
`Laplace` implementation and add the `ScalarTransportFlux`. 

In [169]:
var Op = new DifferentialOperator(1, 0, 1, QuadOrderFunc.Linear(), "u", "R1");

Op.EquationComponents["R1"].Add(new Laplace()); // adding Laplace
Op.EquationComponents["R1"].Add(new ScalarTransportFlux(laxFriedrichsFlux)); // adding Advection
Op.EquationComponents["R1"].Add(new SourceTerm()); // adding source term
Op.TemporalOperator = new ConstantTemporalOperator(Op);
Op.IsLinear = true;
Op.Commit();

## Find the time step

## Performing Temporal Integration

In [170]:

SinglePhaseField TimeIntegrate(SinglePhaseField u0, double EndTime, double dt, TimeSteppingScheme tsScheme=TimeSteppingScheme.ESDIRK_64) {
    var u1 = u0.CloneAs();
    var Timestepper = new XdgTimestepping(Op, new DGField[] { u1 }, new DGField[] { new SinglePhaseField(u0.Basis)}, 
                            tsScheme);

    int NoOfTimeteps = (int) (EndTime/dt);
    for(int i = 0; i < NoOfTimeteps; i++)
        Timestepper.Solve(dt*i, dt);
    
    return u1;
}

## Performing a temporal convergence study

In [171]:
double uExEq(double[] X) {
   return MyGlobals.uExact(X,2.0);
}

loop over different grids (`nCells`) and `TimesteppingSchemes` 

In [184]:
//var dtS = new double[] {1,0.5,0.25,0.125,0.0625,0.03175,0.015875,0.0079375};
var dtS = new double[] {1,0.5,0.25,0.125,0.0625};
var NCells = new double[] {4,8,16};
var tsSchemes= new TimeSteppingScheme[] {TimeSteppingScheme.ESDIRK_64,TimeSteppingScheme.ESDIRK_32};
// var NCells = new double[] {4,8,16,32,64,128};
// var dtS = new double[] {1,0.5};
// var tsSchemes= new TimeSteppingScheme[] {TimeSteppingScheme.ESDIRK_64};
var results = new List<Tuple<string,double[]>>();

//Grid and initial value
foreach(int nCells in NCells){
    var grid = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(0.0,2.0,nCells+1), GenericBlas.Linspace(0.0,2.0,nCells+1));
    var u0 = new SinglePhaseField(new Basis(grid, 2), "u");
    var u1 = new SinglePhaseField(new Basis(grid, 2), "u");
    u0.ProjectField((double[] X) => MyGlobals.uExact(X,0));

    for(int ischeme=0;ischeme<tsSchemes.Length;ischeme++){
        var scheme=tsSchemes[ischeme];
        var solutions = new List<SinglePhaseField>();
        Console.WriteLine("   Starting Compuation: " + scheme.ToString()+": nCells=" +grid.NumberOfCells);
        for(int i = 0; i < dtS.Length; i++) {
            Console.WriteLine("   --------------  computing dt: " + dtS[i]);
            u1 = TimeIntegrate(u0, 2, dtS[i],scheme);

            //use Exact solution
            //u1.ProjectField((double[] X) => MyGlobals.uExact(X,2.0));
            //Tecplot("u1." + i, u1);
            solutions.Add(u1);
        }
        //######## some debugging ####################
        //compute res
        var Ev= Op.GetEvaluatorEx(new List<DGField>(){u1},null,new CoordinateMapping(u1));
        var residual=u1.CloneAs(); residual.Identification="residual";
        Ev.Evaluate(1.0,0.0,new CoordinateVector(residual));
        
        var diff=u1.CloneAs(); diff.Identification="diff_ex";
        diff.ProjectField((double[] X) => MyGlobals.uExact(X,2.0));
        diff.Acc(-1.0,u1);

        var uex=u1.CloneAs(); uex.Identification="u_ex";
        uex.ProjectField((double[] X) => MyGlobals.uExact(X,2.0));

        Tecplot("u1." + ((ischeme+1)*100+nCells), new DGField[] {u1,residual,diff,uex});
        //Computing the error against the exact solution:
        double[] ErrEx = new double[solutions.Count];
        for(int i = 0; i < ErrEx.Length; i++) {
            ErrEx[i] = solutions[i].L2Error(uExEq);
            //ErrEx[i]=residual.L2Norm();
        }
        results.Add(new Tuple<string,double[]>(scheme.ToString()+ ": nCells=" +grid.NumberOfCells,ErrEx));
    }
}



   Starting Compuation: ESDIRK_64: nCells=16
   --------------  computing dt: 1
   --------------  computing dt: 0.5
   --------------  computing dt: 0.25
   --------------  computing dt: 0.125
   --------------  computing dt: 0.0625
Writing output file c:\experimental\public\doc\handbook\AdvectionDiffusionSolver\u1.104...
done.
   Starting Compuation: ESDIRK_32: nCells=16
   --------------  computing dt: 1
   --------------  computing dt: 0.5
   --------------  computing dt: 0.25
   --------------  computing dt: 0.125
   --------------  computing dt: 0.0625
Writing output file c:\experimental\public\doc\handbook\AdvectionDiffusionSolver\u1.204...
done.
   Starting Compuation: ESDIRK_64: nCells=64
   --------------  computing dt: 1
   --------------  computing dt: 0.5
   --------------  computing dt: 0.25
   --------------  computing dt: 0.125
   --------------  computing dt: 0.0625
Writing output file c:\experimental\public\doc\handbook\AdvectionDiffusionSolver\u1.108...
done.
   Star

plot results

In [185]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Blue,LineColors.Black,LineColors.Black, LineColors.Red,LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [186]:
var gp = new Gnuplot();
double[] ts = dtS.Take(dtS.Length).Select(x => (double)x).ToArray();
double[] ts1 = ts.Take(dtS.Length - 1).ToArray();
for(int iRes=0;iRes<results.Count;iRes++){
    var pair= results[iRes];
    gp.PlotLogXLogY(ts, pair.Item2,  title:pair.Item1, format:GetFormat(iRes));
}
gp.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 1000 
 
 
 
 
 10000 
 
 
 
 
 0.01 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 ESDIRK64: nCells=16 
 
 
 ESDIRK 6 4: nCells=16 
 
 
 
 
 
 
 
 
 
 
 
 ESDIRK32: nCells=16 
 
 
 ESDIRK 3 2: nCells=16 
 
 
 
 
 
 
 
 
 
 
 
 ESDIRK64: nCells=64 
 
 
 ESDIRK 6 4: nCells=64 
 
 
 
 
 
 
 
 
 
 
 
 ESDIRK32: nCells=64 
 
 
 ESDIRK 3 2: nCells=64 
 
 
 
 
 
 
 
 
 
 
 
 ESDIRK64: nCells=256 
 
 
 ESDIRK 6 4: nCells=256 
 
 
 
 
 
 
 
 
 
 
 
 ESDIRK32: nCells=256 
 
 
 ESDIRK 3 2: nCells=256